In [42]:

from app.dotenv import base_dir, data_dir
# from app.model.model import  Model
# import os
import pandas as pd
import json

from pandas import DataFrame
from datasets import load_dataset, Dataset, DatasetDict,concatenate_datasets
# import numpy as np
# # set the max columns to none
# pd.set_option('display.max_columns', None)
# # set the max columns to none
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 500)
# print('dotenv params:', {os.getenv('PARAM1')})
# print('base_dir:', base_dir)
print('data_dir:', data_dir)

# # model = Model()
# # print('model:', model)



data_dir: /home/yishai/Desktop/ivrit_ai_llm/data


#### datasets

I verified all whisper-training are in crowd-transcribe-v4

both have duplicates

In [43]:
path_dataset1 = f'{data_dir}/audio-transcripts'
info_dataset1 =  f'''
dict_keys(['train'])
features: ['source', 'episode', 'uuid', 'text', 'attrs'],
num_rows: 2183042
'''

path_dataset2 = f'{data_dir}/whisper-training'
info_dataset2 =  f'''
dict_keys(['train', 'test'])
features: ['uuid', 'audio', 'orig_text', 'text'],
num_rows: 63674
'''




path_dataset3 = f'{data_dir}/crowd-transcribe-v4'
info_dataset3 =  f'''
dict_keys(['train', 'test'])
features: ['uuid', 'audio', 'orig_sentence', 'sentence', 'is_retranscribe'],
num_rows: 169376
'''
# print('dataset3:', path_dataset3)


print(f'dataset1:, f{path_dataset1}, {info_dataset1}')
print(f'dataset2:, f{path_dataset2}, {info_dataset2}')
print(f'dataset3:, f{path_dataset3}, {info_dataset3}')

print('-'*100)

path_dataset = path_dataset1

print('selected dataset:', path_dataset)

dataset1:, f/home/yishai/Desktop/ivrit_ai_llm/data/audio-transcripts, 
dict_keys(['train'])
features: ['source', 'episode', 'uuid', 'text', 'attrs'],
num_rows: 2183042

dataset2:, f/home/yishai/Desktop/ivrit_ai_llm/data/whisper-training, 
dict_keys(['train', 'test'])
features: ['uuid', 'audio', 'orig_text', 'text'],
num_rows: 63674

dataset3:, f/home/yishai/Desktop/ivrit_ai_llm/data/crowd-transcribe-v4, 
dict_keys(['train', 'test'])
features: ['uuid', 'audio', 'orig_sentence', 'sentence', 'is_retranscribe'],
num_rows: 169376

----------------------------------------------------------------------------------------------------
selected dataset: /home/yishai/Desktop/ivrit_ai_llm/data/audio-transcripts


load dataset into df (drop audio if exists)

In [48]:
def load_hf_dataset(path_dataset,cols_to_remove = 'audio'):
    # Load the Huggingface dataset
    dataset = load_dataset(path_dataset)
    # dataset1 = dataset1['train']  
    print(path_dataset)
    print(dataset.keys())
    # Adjust according to your dataset split
    dataset = concatenate_datasets([dataset[key] for key in dataset.keys()])
    
    if cols_to_remove in dataset.features:
        dataset = dataset.remove_columns(cols_to_remove)
    print(dataset)
    return dataset

dataset = load_hf_dataset(path_dataset)
uuid = "929/2018.10.11 פרופ' נועם מזרחי/156"
results = [entry for entry in dataset if uuid in entry['uuid']]
results


/home/yishai/Desktop/ivrit_ai_llm/data/audio-transcripts
dict_keys(['train'])
Dataset({
    features: ['source', 'episode', 'uuid', 'text', 'attrs'],
    num_rows: 2183042
})


[]

save to csv and verify file saved correctly

In [4]:
def save_into_df(chunk, path_dataset: str):
    df = pd.DataFrame(chunk)
    print(f'df.columns:{df.columns}')
    
    
    # replace '' with None for corect csv export
    df = df.replace('', None)
    if 'attrs' in df.columns:
        df['attrs'] = df['attrs'].apply(lambda x: json.dumps(x))

    df.to_csv(f'{path_dataset}.csv', index=False)
    df1 = pd.read_csv(f'{path_dataset}.csv')

    ddf = df[~(df.isnull().sum(axis=1).astype(bool))]
    ddf1 = df1[~(df1.isnull().sum(axis=1).astype(bool))]
    if (ddf1==ddf).all().all():
        print('df and df1 are equal')
        del df,df1,ddf,ddf1


for dataset>100000: save a subset (chunk) of dataset and continue with that

In [5]:

def load_dataset_from_path(path_dataset: str,cols_to_remove = 'audio') -> DataFrame:
    dataset = load_hf_dataset(path_dataset,cols_to_remove)
    # # Load the Huggingface dataset
    # dataset = load_dataset(path_dataset)
    # # dataset1 = dataset1['train']  
    # print(path_dataset)
    # print(dataset.keys())
    # # Adjust according to your dataset split
    # dataset = concatenate_datasets([dataset[key] for key in dataset.keys()])
    # print(dataset)
    # ------------------------------------------------
    # chunk to chunks if needed
    num_shards = 1 + dataset.num_rows//200000
    print(num_shards)
    for i in range(num_shards):
        chunk = dataset.shard(num_shards=num_shards, index=i)
        # print(chunk)
        # chunk.save_to_disk(f'{path_dataset}_chunk{i}')
        # path_dataset = f'{path_dataset}_chunk{i}'
        print('_'*150)
        print('after chunking:')
        
        
        print(chunk)
        # remove dataset column
        chunk_path = f'{path_dataset}_chunk{i}' if num_shards > 1 else path_dataset
        save_into_df(chunk, chunk_path)
        

        
    
load_dataset_from_path(path_dataset)





/home/yishai/Desktop/ivrit_ai_llm/data/audio-transcripts
dict_keys(['train'])
Dataset({
    features: ['source', 'episode', 'uuid', 'text', 'attrs'],
    num_rows: 2183042
})
11
______________________________________________________________________________________________________________________________________________________
after chunking:
Dataset({
    features: ['source', 'episode', 'uuid', 'text', 'attrs'],
    num_rows: 198459
})
df.columns:Index(['source', 'episode', 'uuid', 'text', 'attrs'], dtype='object')
df and df1 are equal
______________________________________________________________________________________________________________________________________________________
after chunking:
Dataset({
    features: ['source', 'episode', 'uuid', 'text', 'attrs'],
    num_rows: 198459
})
df.columns:Index(['source', 'episode', 'uuid', 'text', 'attrs'], dtype='object')
df and df1 are equal
____________________________________________________________________________________________

compare whisper-training and crowd-transcribe-v4 

check if there are any missing values in crowd-transcribe-v4

In [28]:
def remove_duplicate_uuid(df: pd.DataFrame, col: str = 'uuid') -> pd.DataFrame:
    # find the duplicte ocurence in the df1  - drop all except first occurence
    df.drop(df.loc[df[col].duplicated()].index, inplace=True)
    # print(len(df.loc[df[col].duplicated(keep=False)].index))
    print(f'len(df):{len(df)}')
    return df

def get_common_values(df1, df2, col1, col2):
    df1_common_values = df1[col1].isin(df2[col2])#true\false
    df1_common_index = df1.index[df1_common_values]#row index
    df1_values = df1[col1].loc[df1_common_index]#the actual value
    return df1_values

df2 = pd.read_csv(f'{data_dir}/whisper-training.csv').reset_index(drop=True)
df1 = pd.read_csv(f'{data_dir}/crowd-transcribe-v4.csv').reset_index(drop=True)
df1 = remove_duplicate_uuid(df1)
df2 = remove_duplicate_uuid(df2)
col1 = 'uuid'
col2 = 'uuid'

df1_values = get_common_values(df1, df2, col1, col2)
df2_values = get_common_values(df2, df1, col2, col1)

print(len(df1_values), len(df2_values)) 

len(df):157405
len(df):62662
57941 57941


add metadata from audio-transcripts into crowd-transcribe-v4

In [38]:

print('df1_values')
df1 = pd.read_csv(f'{data_dir}/crowd-transcribe-v4.csv').reset_index(drop=True)
df1 = remove_duplicate_uuid(df1)
cols = ['attrs','source','episode']
for col in cols:#cols to copy
    df1[col]  = None
print('_'*150)   
for chunk_i in range(11):
    print(chunk_i)
    df2 = pd.read_csv(f'{data_dir}/audio-transcripts_chunk{chunk_i}.csv').reset_index(drop=True)
    df2 = remove_duplicate_uuid(df2)
    
    df1_values = get_common_values(df1, df2, col1, col2)
    df2_values = get_common_values(df2, df1, col2, col1)
    df1_values.sort_values(inplace=True)
    df2_values.sort_values(inplace=True)
    
    print('uuids are equal:', (df1.loc[df1_values.index, 'uuid'] == df2.loc[df2_values.index, 'uuid'].values).all())

    for col in cols:#cols to copy
        df1.loc[df1_values.index, col] = df2.loc[df2_values.index, col].values
    print('_'*150)

len(df):157405
0
len(df):198459
uuids are equal: True
1
len(df):198459
uuids are equal: True
2
len(df):198459
uuids are equal: True
3
len(df):198459
uuids are equal: True
4
len(df):198458
uuids are equal: True
5
len(df):198458
uuids are equal: True
6
len(df):198458
uuids are equal: True
7
len(df):198458
uuids are equal: True
8
len(df):198458
uuids are equal: True
9
len(df):198458
uuids are equal: True
10
len(df):198458
uuids are equal: True


save final result to file

In [50]:

df = df1.copy()
df.to_csv(f'{data_dir}/crowd-transcribe-v4-enriched.csv', index=False)
df1 = pd.read_csv(f'{data_dir}/crowd-transcribe-v4-enriched.csv')

ddf = df[~(df.isnull().sum(axis=1).astype(bool))]
ddf1 = df1[~(df1.isnull().sum(axis=1).astype(bool))]
if (ddf1==ddf).all().all():
    print('df and df1 are equal')
    del df1,ddf,ddf1#df,

df and df1 are equal


there are 39317 mising uuid's from audio-transcripts sp not all rows with atributes

In [54]:
df.isnull().sum()

# ind = df.loc[df['attrs'].isnull()].index

# df['uuid'].loc[ind[0]]

uuid                   0
orig_sentence          1
sentence              52
is_retranscribe        0
attrs              39317
source             39317
episode            39317
dtype: int64